In [28]:
import sys

import eikon as ek
import cufflinks as cf
import pandas as pd
from tqdm import tqdm

sys.path.append('../')
import src.eikon_utils as ek_utils
from src.constants import MANUAL_ASSET_CODE_MAPPING

cf.go_offline()

In [5]:
# Connect to the Eikon
ek_utils.connect('../../data/configs/refinitiv-data.config_uts_3.json')


        Please, log in with the following credentials to the Refinitiv WorkSpace first. When logged in - press enter.


        Username: UTS.Finance3@uts.edu.au

        Password: copied to your system clipboard. Use CTRL+V on Windows or CMD+V on Mac to fill in the WorkSpace
    


In [43]:
history_index_df = ek_utils.load_index_constituents_historical('0#.DJI')
# Each month has a list of constituents. Transform data frame to have (Date, Instrument) pairs
history_index_df = history_index_df.explode('constituents')
history_index_df.rename({'constituents': 'Instrument'}, axis=1, inplace=True)
del history_index_df['num_companies']
# Remove missing values
history_index_df = history_index_df[
    (history_index_df['Instrument'].notna())
    &
    (history_index_df['Instrument'].str.len()>0)    
]

In [44]:
# For each instrument append an asset code
# Load in batches, because Eikon may not return correct results otherwise

BATCH = 10
list_to_load = list(history_index_df['Instrument'].unique())

index_mapping_df = list()
for b_start in tqdm(range(0, len(list_to_load), BATCH)):
    map_df = ek.get_symbology(list_to_load[b_start: b_start+BATCH])
    index_mapping_df.append(map_df.copy())
    del map_df
index_mapping_df = pd.concat(index_mapping_df)
index_mapping_df = index_mapping_df.rename_axis('Instrument').reset_index()
index_mapping_df = index_mapping_df[['Instrument', 'OAPermID']]

# Add Manual correction
index_mapping_df = pd.concat([
    index_mapping_df,
    pd.DataFrame(
        list(MANUAL_ASSET_CODE_MAPPING.items()), 
        columns=['Instrument', 'OAPermID']).explode('OAPermID')], 
        ignore_index=True)


# Append asset IDs to the main instrument mapping
history_index_df = pd.merge(
    history_index_df,
    index_mapping_df.groupby("Instrument", as_index=False)['OAPermID'].agg(lambda v: list(set(v))), on='Instrument', how='left')
history_index_df.rename({'OAPermID': 'assetCode'}, axis=1, inplace=True)

100%|██████████| 6/6 [00:02<00:00,  2.54it/s]


In [45]:
# For CSCO.OQ - there are two assetCodes (pre + post mergers) [4295905952, 5080018615]. Remove PostMerge asset
history_index_df.loc[:, 'assetCode'] = history_index_df['assetCode'].apply(lambda v: [z for z in v if z != '5080018615'])

In [49]:
history_index_df.explode('assetCode').to_csv(
    '../../data/general_dumps/history_index_df', 
    index=False)